In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
from sklearn.preprocessing import LabelBinarizer

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
datapath = r"C:\Users\ASUS\Desktop\Sports-type classifier\data"
outputmodel = r"C:\Users\ASUS\Desktop\Sports-type classifier\project-1\videoclassificationmodel"
outputlabelbinarizer = r"C:\Users\ASUS\Desktop\Sports-type classifier\project-1\videoclassificationbinarizer"

In [6]:
Sports_Labels = set(['boxing', 'swimming', 'table_tennis'])
print("Images is being loaded....")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image)
    labels.append(label)
    

Images is being loaded....


In [7]:
data = np.array(data)
labels = np.array(labels)

#hot encoded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [8]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data,labels,test_size = 0.25, stratify = labels, random_state = 1)

In [9]:
training_augmentation = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validation_augmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype = 'float32')
training_augmentation.mean = mean
validation_augmentation.mean = mean

In [10]:
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Flatten
from keras.layers.core import Dense 
from keras.layers.core import Dropout
from keras.models import Model

In [11]:
baseModel = ResNet50(weights='imagenet',include_top = False, input_tensor = Input(shape = (244, 244, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7,7))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(512,activation = 'relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation = "softmax")(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

C:\Users\ASUS\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [12]:
epoch = 25

In [13]:
from keras.optimizers import SGD

In [14]:
opt = SGD(lr = 0.0001, momentum = 0.9, decay = 1e-4/epoch)

In [15]:
model.compile(loss = "categorical_crossentropy",
             optimizer = opt,
             metrics = ['accuracy'])

In [16]:
history = model.fit_generator(training_augmentation.flow(X_train, Y_train, batch_size = 32),
                             steps_per_epoch = len(X_train) // 32,
                             validation_data = validation_augmentation.flow(X_test, Y_test),
                             validation_steps = len(X_test) // 32,
                             epochs = epoch)

Epoch 1/25
49/49 [==============================] - 511s 10s/step - loss: 1.3144 - accuracy: 0.3878 - val_loss: 0.8915 - val_accuracy: 0.5801
Epoch 2/25
49/49 [==============================] - 463s 9s/step - loss: 0.8685 - accuracy: 0.6194 - val_loss: 0.6993 - val_accuracy: 0.7617
Epoch 3/25
49/49 [==============================] - 460s 9s/step - loss: 0.6581 - accuracy: 0.7189 - val_loss: 0.4851 - val_accuracy: 0.8045
Epoch 4/25
49/49 [==============================] - 461s 9s/step - loss: 0.5285 - accuracy: 0.7768 - val_loss: 0.5460 - val_accuracy: 0.8513
Epoch 5/25
49/49 [==============================] - 456s 9s/step - loss: 0.4654 - accuracy: 0.8256 - val_loss: 0.3163 - val_accuracy: 0.8635
Epoch 6/25
49/49 [==============================] - 462s 9s/step - loss: 0.3968 - accuracy: 0.8508 - val_loss: 0.4671 - val_accuracy: 0.9002
Epoch 7/25
49/49 [==============================] - 447s 9s/step - loss: 0.3994 - accuracy: 0.8606 - val_loss: 0.5225 - val_accuracy: 0.8737
Epoch 8/25
4

In [17]:
import pickle

In [18]:
model.save(outputmodel)
lbinarizer = open(r"C:\Users\ASUS\Desktop\Sports-type classifier\project-1\videoclassificationbinarizer.pickle","wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()